In [10]:
# Colab setup ------------------
import os, sys, subprocess
# ------------------------------

import numpy as np
import scipy.integrate

import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting

import GATA6_response_model

# Set to True to have fully interactive plots with Python;
# Set to False to use pre-built JavaScript-based plots
interactive_python_plots = False
notebook_url = "localhost:8888"

bokeh.io.output_notebook()

def callback(attr, old, new):
    for i in range(len(gata6)):
        _, _, ss = GATA6_response_model.RunDeterministicResponse(GATA6=i * 1000,
                                                            inhibited_transcription_rate = [5e-8,5e-8],
                                                            activated_transcription_rate = [pFOXA2_GATA6_tr_slider.value, pFOXF1_GATA6_tr_slider.value],
                                                            double_activated_transcription_rate = [pFOXA2_FOXA2_GATA6_tr_slider.value, pFOXF1_FOXF1_GATA6_tr_slider.value],
                                                            triple_activated_transcription_rate = [pFOXA2_p3_tr_slider.value, pFOXA2_p3_tr_slider.value],
                                                            P_act_binding=[pFOXA2_to_pFOXA2_GATA6.value, pFOXF1_to_pFOXF1_GATA6.value],
                                                            P_inh_binding=[pFOXA2_to_pFOXA2_FOXF1.value, pFOXF1_to_pFOXF1_FOXA2.value],
                                                            P1_act_binding=[pFOXA2_GATA6_to_p2.value, pFOXF1_GATA6_to_p2.value],
                                                            P1_inh_unbinding=[pFOXA2_FOXF1_to_pFOXA2.value, pFOXF1_FOXA2_to_pFOXF1.value],
                                                            P1_act_unbinding=[pFOXA2_GATA6_to_pFOXA2.value, pFOXF1_GATA6_to_pFOXF1.value],
                                                            P2_act_unbinding=[p2_to_pFOXA2_GATA6.value, p2_to_pFOXF1_GATA6.value],
                                                            P2_act_binding=[pFOXA2_p2_to_pFOXA2_p3.value, pFOXF1_p2_to_pFOXF1_p3.value],
                                                            P3_act_unbinding=[pFOXA2_p3_to_pFOXA2_p2.value, pFOXF1_p3_to_pFOXF1_p2.value])
        foxa2s[i] = ss[-1,2]
        foxf1s[i] = ss[-1,3]
    cds.data["foxa2"] = foxa2s
    cds.data["foxf1"] = foxf1s

pFOXA2_GATA6_tr_slider = bokeh.models.Slider(
    title="pFOXA2_GATA6 activated transcription rate", start=0, end=1, step=0.001, value=0.005, width=150
)

pFOXA2_FOXA2_GATA6_tr_slider = bokeh.models.Slider(
    title="pFOXA2_GATA6_FOXA2 activated transcription rate", start=0, end=1, step=0.001, value=0.05, width=150
)
pFOXF1_GATA6_tr_slider = bokeh.models.Slider(
    title="pFOXF1_GATA6 activated transcription rate", start=0, end=1, step=0.001, value=0.005, width=150
)

pFOXF1_FOXF1_GATA6_tr_slider = bokeh.models.Slider(
    title="pFOXF1_GATA6_FOXF1 activated transcription rate", start=0, end=1, step=0.001, value=0.025, width=150
)
pFOXA2_p3_tr_slider = bokeh.models.Slider(
    title="pFOXA2_GATA6_FOXA2_FOXA2 activated transcription rate", start=0, end=1, step=0.001, value=0.1, width=150
)
pFOXF1_p3_tr_slider = bokeh.models.Slider(
    title="pFOXF1_GATA6_FOXF1_FOXF1 activated transcription rate", start=0, end=1, step=0.001, value=0.05, width=150
)

pFOXA2_to_pFOXA2_GATA6 = bokeh.models.Slider(
    title="pFOXA2 -> pFOXA2_GATA6 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
pFOXF1_to_pFOXF1_GATA6 = bokeh.models.Slider(
    title="pFOXF1 -> pFOXF1_GATA6 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
pFOXA2_GATA6_to_pFOXA2 = bokeh.models.Slider(
    title="pFOXA2_GATA6 -> pFOXA2 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)
pFOXF1_GATA6_to_pFOXF1 = bokeh.models.Slider(
    title="pFOXF1_GATA6 -> pFOXF1 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)

pFOXA2_to_pFOXA2_FOXF1 = bokeh.models.Slider(
    title="pFOXA2 -> pFOXA2_FOXF1 binding rate", start=0, end=1000, step=1.0, value=50.0, width=150
)
pFOXF1_to_pFOXF1_FOXA2 = bokeh.models.Slider(
    title="pFOXF1 -> pFOXF1_FOXA2 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
pFOXA2_FOXF1_to_pFOXA2 = bokeh.models.Slider(
    title="pFOXA2_FOXF1 -> pFOXA2 binding rate", start=0, end=1000, step=1.0, value=25.0, width=150
)
pFOXF1_FOXA2_to_pFOXF1 = bokeh.models.Slider(
    title="pFOXF1_FOXA2 -> pFOXF1 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)


pFOXA2_GATA6_to_p2 = bokeh.models.Slider(
    title="pFOXA2_GATA6 -> P2 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
pFOXF1_GATA6_to_p2 = bokeh.models.Slider(
    title="pFOXF1_GATA6 -> P2 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)

p2_to_pFOXA2_GATA6 = bokeh.models.Slider(
    title="P2 -> pFOXA2_GATA6 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)
p2_to_pFOXF1_GATA6 = bokeh.models.Slider(
    title="P2 -> pFOXF1_GATA6 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)

pFOXA2_p2_to_pFOXA2_p3 = bokeh.models.Slider(
    title="pFOXA2_p2 -> pFOXA2_p3 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
pFOXF1_p2_to_pFOXF1_p3 = bokeh.models.Slider(
    title="pFOXF1_p2 -> pFOXF1_p3 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
pFOXA2_p3_to_pFOXA2_p2 = bokeh.models.Slider(
    title="pFOXA2_p3 -> pFOXA2_p2 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)
pFOXF1_p3_to_pFOXF1_p2 = bokeh.models.Slider(
    title="pFOXF1_p3 -> pFOXF1_p2 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)
# Set up time points
gata6_max = 5
gata6 = np.linspace(0, gata6_max, 6)
t = np.linspace(0, 10000, 10000)

foxa2s = np.zeros(len(gata6))
foxf1s = np.zeros(len(gata6))

# Build s, taking slider values are parameters.


for i in range(len(gata6)):
    _, _, ss = GATA6_response_model.RunDeterministicResponse(GATA6=i * 1000 + 1,
                                                            inhibited_transcription_rate = [5e-8,5e-8],
                                                            activated_transcription_rate = [pFOXA2_GATA6_tr_slider.value, pFOXF1_GATA6_tr_slider.value],
                                                            double_activated_transcription_rate = [pFOXA2_FOXA2_GATA6_tr_slider.value, pFOXF1_FOXF1_GATA6_tr_slider.value],
                                                            triple_activated_transcription_rate = [pFOXA2_p3_tr_slider.value, pFOXF1_p3_tr_slider.value],
                                                            P_act_binding=[pFOXA2_to_pFOXA2_GATA6.value, pFOXF1_to_pFOXF1_GATA6.value],
                                                            P_inh_binding=[pFOXA2_to_pFOXA2_FOXF1.value, pFOXF1_to_pFOXF1_FOXA2.value],
                                                            P1_act_binding=[pFOXA2_GATA6_to_p2.value, pFOXF1_GATA6_to_p2.value],
                                                            P1_inh_unbinding=[pFOXA2_FOXF1_to_pFOXA2.value, pFOXF1_FOXA2_to_pFOXF1.value],
                                                            P1_act_unbinding=[pFOXA2_GATA6_to_pFOXA2.value, pFOXF1_GATA6_to_pFOXF1.value],
                                                            P2_act_unbinding=[p2_to_pFOXA2_GATA6.value, p2_to_pFOXF1_GATA6.value],
                                                            P2_act_binding=[pFOXA2_p2_to_pFOXA2_p3.value, pFOXF1_p2_to_pFOXF1_p3.value],
                                                            P3_act_unbinding=[pFOXA2_p3_to_pFOXA2_p2.value, pFOXF1_p3_to_pFOXF1_p2.value])
    foxa2s[i] = ss[-1,2]
    foxf1s[i] = ss[-1,3]


# Place the data in a ColumnDataSource
cds = bokeh.models.ColumnDataSource(dict(gata6=gata6, foxa2=foxa2s, foxf1=foxf1s))

# Build the plot
p = bokeh.plotting.figure(
    frame_height=400,
    frame_width=400,
    x_axis_label="GATA6 / 1000",
    y_axis_label="GATA6 Response",
    x_range=[0, 5],
    y_range=[-0.02, 1000],
)
p.line(source=cds, x="gata6", y="foxa2", line_width=2, color='red', legend_label="FOXA2");
p.line(source=cds, x="gata6", y="foxf1", line_width=2, color='green', legend_label="FOXF1")

pFOXA2_GATA6_tr_slider.on_change("value", callback)
pFOXA2_FOXA2_GATA6_tr_slider.on_change("value", callback)
pFOXF1_GATA6_tr_slider.on_change("value", callback)
pFOXF1_FOXF1_GATA6_tr_slider.on_change("value", callback)
pFOXA2_p3_tr_slider.on_change("value", callback)
pFOXF1_p3_tr_slider.on_change("value", callback)

pFOXA2_to_pFOXA2_GATA6.on_change("value", callback)
pFOXF1_to_pFOXF1_GATA6.on_change("value", callback)
pFOXA2_to_pFOXA2_FOXF1.on_change("value", callback)
pFOXF1_to_pFOXF1_FOXA2.on_change("value", callback)

pFOXA2_GATA6_to_pFOXA2.on_change("value", callback)
pFOXF1_GATA6_to_pFOXF1.on_change("value", callback)
pFOXA2_FOXF1_to_pFOXA2.on_change("value", callback)
pFOXF1_FOXA2_to_pFOXF1.on_change("value", callback)

pFOXA2_GATA6_to_p2.on_change("value", callback)
pFOXF1_GATA6_to_p2.on_change("value", callback)
p2_to_pFOXA2_GATA6.on_change("value", callback)
p2_to_pFOXF1_GATA6.on_change("value", callback)

pFOXA2_p2_to_pFOXA2_p3.on_change("value", callback)
pFOXF1_p2_to_pFOXF1_p3.on_change("value", callback)
pFOXA2_p3_to_pFOXA2_p2.on_change("value", callback)
pFOXF1_p3_to_pFOXF1_p2.on_change("value", callback)

layout = bokeh.layouts.column(
    bokeh.layouts.row(p, bokeh.layouts.column(pFOXA2_GATA6_tr_slider,
                        pFOXF1_GATA6_tr_slider,
                        pFOXA2_FOXA2_GATA6_tr_slider,
                        pFOXF1_FOXF1_GATA6_tr_slider,
                        pFOXA2_p3_tr_slider,
                        pFOXF1_p3_tr_slider)),
    bokeh.layouts.row(
    bokeh.layouts.column(pFOXA2_to_pFOXA2_GATA6,
                        pFOXF1_to_pFOXF1_GATA6,
                        pFOXA2_GATA6_to_pFOXA2,
                        pFOXF1_GATA6_to_pFOXF1,),
    bokeh.models.Spacer(width=150),
    bokeh.layouts.column(
                        pFOXA2_to_pFOXA2_FOXF1,
                        pFOXF1_to_pFOXF1_FOXA2,
                        pFOXA2_FOXF1_to_pFOXA2,
                        pFOXF1_FOXA2_to_pFOXF1)),
    bokeh.layouts.row(
    bokeh.layouts.column(pFOXA2_GATA6_to_p2,
                        pFOXF1_GATA6_to_p2,
                        p2_to_pFOXA2_GATA6,
                        p2_to_pFOXF1_GATA6),
    bokeh.models.Spacer(width=150),
    bokeh.layouts.column(pFOXA2_p2_to_pFOXA2_p3,
                        pFOXF1_p2_to_pFOXF1_p3,
                        pFOXA2_p3_to_pFOXA2_p2,
                        pFOXF1_p3_to_pFOXF1_p2))
)

def app(doc):
    doc.add_root(layout)

Loading BokehJS ...

In [11]:
interactive_python_plots = True
if interactive_python_plots:
    bokeh.io.show(app, notebook_url=notebook_url)